In [3]:
import json
import os
from datetime import datetime, timedelta

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import CSVSearchTool, Tool

In [10]:
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

In [8]:
import yfinance as yf

In [ ]:
LLM = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
CSV_TOOL = CSVSearchTool(csv="./data/stock_data.csv")

In [ ]:
customer_manager = Agent(
    role="Customer Stocks Manager",
    goal="Get the customer question about the stock {ticket} and search for the best answer in the stock data.",
    backstory="You're a customer support agent specialized in stock market data. Your job is to assist customers by providing accurate and relevant information based on their inquiries about stock performance, trends, and other related data.",
    verbose=True,
    llm=LLM,
    max_iter=5,
    tools=[CSV_TOOL],
    allow_delegation=False,
    memory=True
)

In [ ]:
get_customer_wallet = Task(
    description="""Use the customer question about the stock {ticket} in the CSV file.
    Provide if the stock is in the customer's wallet and if so, provide with the mean price he paid and the total numbers of stocks owned.""",
    expected_output="Customer question about the stock.",
    agent=customer_manager
)

In [ ]:
stocket_price_analyst = Agent(
    role="Senior Stock Price Analyst",
    goal="Analyze the stock price data and provide insights on stock performance, trends, and recommendations.",
    backstory="You're a financial analyst specializing in stock market trends and data analysis. Your role is to interpret stock price data and provide actionable insights and recommendations based on your analysis.",
    verbose=True,
    llm=LLM,
    max_iter=5,
    allow_delegation=False,
    memory=True
)

In [ ]:
# Create Yfinance Tool

def fetch_stock_data(ticket: str):
    start = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')
    end = datetime.now().strftime('%Y-%m-%d')
    stock = yf.download(ticket, start=start, end=end)
    return stock

YAHOO_FINANCE_TOOL = Tool(
    name="Yahoo Finance Stock Data Fetcher",
    func=lambda ticket: fetch_stock_data(ticket),
    description="Fetch historical stock price data for a given stock ticker symbol using Yahoo Finance.",
)

In [ ]:
get_stock_price = Task(
    description="Analyze the stock price data for the stock {ticket} and provide insights on its performance, trends, and recommendations.",
    expected_output="Detailed analysis of the stock price data.",
    agent=stocket_price_analyst,
    tools=[YAHOO_FINANCE_TOOL],
)

In [ ]:
news_analyst = Agent(
    role="Market News Analyst",
    goal="Analyze recent market news and provide insights on how it may impact stock performance and trends.",
    backstory="You're a market news analyst specializing in interpreting financial news and its implications on stock markets. Your role is to analyze news articles and provide insights on how they may affect stock performance and trends.",
    verbose=True,
    llm=LLM,
    max_iter=5,
    allow_delegation=False,
    memory=True
)

In [ ]:
search_tool = DuckDuckGoSearchResults(backend="news", num_results=10)

In [ ]:
get_news = Task(
    description=f"""
    Use the search Tool to search news about the stock ticket.
    The currente date is {datetime.now().strftime('%Y-%m-%d')}.
    Compose the results into a detailed analysis of recent market news and its potential impact on the stock.
    """,
    expected_output="Detailed analysis of recent market news and its potential impact on the stock.",
    agent=news_analyst,
    tools=[search_tool],
)

In [ ]:
stock_recommender = Agent(
    role="Chief Stock Recommender",
    goal="Provide stock recommendations based on stock price analysis and recent market news.",
    backstory="You're a stock recommender specialist who synthesizes stock price data analysis and market news to provide well-informed stock recommendations. Your role is to evaluate the insights provided by financial analysts and market news analysts to recommend whether to buy, hold, or sell stocks.",
    verbose=True,
    llm=LLM,
    max_iter=5,
    allow_delegation=True,
    memory=True
)

In [ ]:
recommend_stock = Task(
    description="""
    Based on the analysis of the stock price data and recent market news, provide a comprehensive stock recommendation.
    The recommendation should include whether to buy, hold, or sell the stock, along with a detailed rationale for the recommendation.
    """,
    expected_output="Comprehensive stock recommendation including buy, hold, or sell advice with rationale.",
    agent=stock_recommender,
    context=[get_news, get_customer_wallet, get_stock_price]
)

In [ ]:
copywriter = Agent(
    role="Stock Content Writer",
    goal="Create engaging and informative content based on the stock recommendation provided.",
    backstory="You're a skilled content writer specializing in financial topics, particularly stock market analysis and recommendations. Your role is to transform complex stock recommendations into clear, engaging, and informative content that resonates with a broad audience.",
)